이제까지 X와 Y가 모두 주어진 훈련상황을 가정했습니다. 이번에는 X만 주어진 상태에서 $\hat{Y}$을 추론하는 방법에 관해 서술하겠습니다. 이러한 과정을 추론 또는 $탐색^{search}$라고 부릅니다. 결국 우리가 원하는 것은 단어들 사이에서 최고의 확률을 갖는 $경로^{path}$를 찾는 것입니다.

### Sampling

먼저 떠올릴 수 있는 가장 정확한 방법은 각 time-step별 $\hat{y_t}$을 고릴때 마지막 softmax 계층에서의 확률 분포대로 샘플링하는 것입니다. 그리고 다음 time-step에서 그 선택 $\hat{y_t}$을 기반으로 다음 $\hat{y_{t+1}}$을 도 다시 샘플링하여 최종적으로 EOS가 나올때까지 샘플링을 반복합니다. 이렇게 하면 우리가 원하는 분포에 가장 가까운 형태의 번역이 완성될 것입니다. 

하지만 이러한 방식은 같은 입력에 대해 매번 다른 출력 결과물을 만들어 낼 수 있습니다. 따라서 우리가 원하는 형태의 결과물은 아닙니다.

<br></br>
$$
\hat{y_t} \sim P(y_t|X,\hat{y_{<t}};\theta)
$$
<br></br>

### Greedy Search Algorithm

탐욕 탐색 알고리즘을 기반으로 탐색을 구현해보겠습니다. 즉, 소프트맥스 계층에서 가장 확률값이 큰 인덱스를 뽑아 해당 time-step의 $\hat{y_t}$으로 사용하는 것입니다. 이를 수식으로 나타내면 다음과 같습니다.

<br></br>
$$
\hat{y_t} = argmax_{y \in Y} P(y_t|X,\hat{y_{<t}};\theta)
$$
<br></br>

<br></br>
![](./images/10-6-2-greedy.jpg)
<br></br>

### 파이토치 코드

다음은 Greedy Search Algorithm을 위한 코드입니다. 인코더가 동작하는 부분까지는 완전히 똑같습니다. 다만, 이후 추론을 위한 부분은 teacher forcing을 사용했던 훈련 방식과 달리, 실제 이전 time-step의 출력을 현재 time-step의 입력으로 사용합니다.

In [1]:
def search(self, src, is_greedy = True, max_length = 255):
    mask, x_length = None, None
    
    if isinstance(src, tuple):
        x, x_length = src
        mask = self.generate_mask(x, x_length)
        
    else:
        x = src
        
    batch_size = x.size(0)
    
    emb_src = self.emb_src(x)
    h_src, h_0_tgt = self.encoder((emb_src, x_length))
    h_0_tgt, c_0_tgt = h_0_tgt
    h_0_tgt = h_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size).transpose(0,1).contiguous()
    
    c_0_tgt = c_0_tgt.transpose(0, 1).contiguous().view(batch_size,
                                                        -1,
                                                        self.hidden_size).transpose(0,1).contiguous()
    
    h_0_tgt = (h_0_tgt, c_0_tgt)
    
    ## Fill a vector, which has "batch_size" dimension with BOS value.
    y = x.new(batch_size, 1).zero_() + data_loader.BOS
    is_undone = x.new_ones(batch_size, 1).float()
    decoder_hidden = h_0_tgt
    h_t_tilde, y_hats, indice = None, [], []
    
    ## Repeat a loop while sum of "is_undone" flag is bigger than 0
    ## or current time-step is smaller than maximum length
    
    while is_undone.sum() > 0 and len(indice) < max_length:
        
        ## Unlike training procedure, take the last time-step's output during the inference.
        ## |emb_t| = (batch_size, 1, word_vec_dim)
        emb_t = self.emb_dec(y)
        
        decoder_output, decoder_hidden = self.decoder(emb_t, 
                                                      h_t_tilde, 
                                                      decoder_hidden)
        
        context_vector = self.attn(h_src, decoder_output, mask)
        h_t_tilde = self.tanh(self.concat(torch.cat([decoder_output,
                                                     context_vector], dim = -1)))
        
        ## |y_hat| = (batch_size, 1, output_size)
        y_hat = self.generator(h_t_tilde)
        
        y_hats += [y_hat]
        
        if is_greedy:
            y = torch.topk(y_hat, 1, dim = -1)[1].squeeze(-1)
            
        else:
            ## Take a random sampling baed on the multinoulli distribution
            y = torch.multinomial(y_hat.exp().view(batch_size,-1), 1)
            
        ## Put PAD if the sample is done
        
        ## |y| = (batch_size, 1)
        ## |is_undone| = (batch_size, 1)
        y = y.masked_fill_((1. - is_undone).byte(), data_loader.PAD)
        is_undone = is_undone * torch.ne(y, data_loader.EOS).float()
        
        indice += [y]
    
    ## |y_hats| = (batch_size, length, output_size)
    ## |indice| = (batch_size, length)
    y_hats = torch.cat(y_hats, dim = 1)
    indice = torch.cat(indice, dim = -1)
    
    return y_hats, indice

가끔 너무 어렵거나 훈련 데이터에서 볼 수 없었던 형태의 문장이 인코딩되어 들어오거나, 훈련 데이터가 적어서 디코더가 잘 훈련되어 있지 않으면, 같은 단어를 반복하며 끝이 없는 문장을 뱉어내는 현상이 발생할 수 있습니다. 즉, EOS가 나오지 않는 상황이 발생할 수 있습니다.

<br></br>

|정상|비정상|
|---|----|
|나는 학교에 갑니다.|나는 학교에 학교에 학교에 학교에 학교에 ...|

<br></br>

따라서 우리는 앞의 함수 입력에서 볼 수 있듯이, 최대 가능 문장 길이를 정해주어 끝이 없는 문장이 나오는 경우에 대비합니다.